# Imports

Install new libraries

In [ ]:
!pip install serpapi
!pip install pyPDF2
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=eab43f0ea12ec3a5d636e8f298fedaadb4670482a3cbd025b9f968472da1d29e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
!pip install chromadb
!pip install langchain
!pip install -U langchain-community
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73

Import the libraries

In [ ]:
# libraries for web searching
import serpapi

# libraries to open webpages
from urllib.request import Request, urlopen
from urllib.error import HTTPError, URLError
import io
import requests
import ssl

# libraries for web scraping
from bs4 import BeautifulSoup
import wikipedia
import PyPDF2

# used for Collection
import chromadb
from chromadb.db.base import UniqueConstraintError
from chromadb.utils import embedding_functions

# used for Google Gemini
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import os
import google.generativeai as genai

# Input the UN website link for the gift

**Input your corresponding API keys Google Gemini and SerpApi**

In [ ]:
# API keys
Gemini_API_KEY = "FILL IN YOUR GEMINI API KEY HERE"
SerpApi_API_KEY = "FILL IN YOUR SERPAPI API KEY HERE"

Input the UN website link for the gift. **Make sure to click enter/return after inputting the link.**


In [ ]:
# ask for the link from the UN website
link = input("UN website link for gift given to the UN (Press enter to enter the link into the code): ")

# open the webpage
req = Request(
    url=link,
    headers={'User-Agent': 'Mozilla/5.0'}
)
webpage = urlopen(req).read()
webpage = BeautifulSoup(webpage, 'html.parser')

UN website link for gift given to the UN (Press enter to enter the link into the code): https://www.un.org/ungifts/poseidon-artemision


# Scrape information from the UN website

In [ ]:
results = webpage.find_all("div", class_="panel-panel-inner")

# description
description = results[1].find_all("div", class_="field-type-text-with-summary")
description_text = BeautifulSoup(description[0].text, "lxml").text.replace("\n", "").replace("\xa0", "")

# donation date
year_results = results[1].find_all("div", class_="field-name-field-donation-date")
year_results_text = BeautifulSoup(year_results[0].text, "lxml").text.replace("\xa0", "").split(":")[-1]

# donor country
country_results = results[1].find_all("div", class_="field-name-field-donor")
country_results_text = BeautifulSoup(country_results[0].text, "lxml").text.replace("\xa0", "").split(":")[-1]

# Google search prompts RAG

Allow Google Gemini to create search prompts for further examination of the circumstance for gifting this gift.

In [ ]:
# defining the search prompt questions
questions = [
    "Given " + description_text + ", answer what are possible reasons for this gift being gifted to the UN that is not cultural?",
    "Provide a list of just 5 prompts to search up reasons listed above for  " + country_results_text + " during the exact time of the gifting (" + year_results_text + ")."
]

Configure Google Gemini

In [ ]:
genai.configure(api_key=Gemini_API_KEY)

In [ ]:
# creating the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  safety_settings = {
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }
)

chat_session = model.start_chat(
  history=[
  ] #currently, no chat history
)

In [ ]:
# function get_search_prompts from Gemini's response
def get_search_prompts(response_text):
  '''get_search_prompts(response_text) -> list
  returns a list of string of the suggested prompts from Google Gemini.'''
  # init variables
  search_prompts = []
  end_index = -1

  # loop through until find all prompts
  while True:
    # find the start index of a prompt
    start_index = response_text[end_index+1:].find('"')


    if start_index == -1: # check if there are any more prompts provided by Gemini
      break
    else:
      start_index += end_index+1

    # find the end index of a prompt
    end_index = response_text[start_index+1:].find('"') + start_index + 1

    search_prompts.append(response_text[start_index+1:end_index])# find the prompt and add it to the search prompt list

  #return the search prompts
  return search_prompts



Ask Google Gemini to provide search prompts and save the search prompts for further use.

In [ ]:
# asking Google Gemini for search prompts
response0 = chat_session.send_message(str(questions[0]))
response1 = chat_session.send_message(str(questions[1]))

In [ ]:
# save the search prompts
search_prompts = get_search_prompts(response1.text)

# Get informtion from websites found via the suggested prompts

Use the search prompts provided by Google Gemini to search for websites that would be able to provide more information on to the situation of the gifting.

In [ ]:
# get the search prompts
search_prompts = search_prompts[:5] # makes sure that there is only 5 search prompts
priority_keywords = [] #keywords

# implmented as sets to make sure there are no duplicates
general_other_articles = set() # these will be links
general_priority_articles = set() # these will be links
wikipedia_articles = set() # use the article names to scrape information from Wikipedia
search_results = []

for search_query in search_prompts:
  params = {
    "q": search_query,
    "hl": "en",
    "gl": "us",
    "google_domain": "google.com",
    "api_key": SerpApi_API_KEY,
  }
  search_results = serpapi.search(params)

  for result_webpage in search_results["organic_results"]:
    # if it is a wikipedia site save it seperately
    if result_webpage["source"].lower() == "wikipedia":
      wikipedia_articles.add(result_webpage["title"])
    else:
      # check if it is a priority website
      if (result_webpage["source"].lower() in ["britannica"]) or ("un.org" in result_webpage["link"]) or (".gov" in result_webpage["link"]) or len([0 for keyword in priority_keywords if result_webpage["source"].find(keyword) > 0]):
        general_priority_articles.add(result_webpage["link"])
      else:
        general_other_articles.add(result_webpage["link"])


Define a function to help with scraping information form websites

In [ ]:

# set up SSL context to allow legacy TLS versions
ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT

def get_paragraphs(webpage_link):
  '''get_paragraphs(webpage_link) -> list, str
  returns a list of string of the paragraphs from this webpage, and the string version of the file type'''
  # open the webpage
  req = Request(
      url=webpage_link,
      headers={'User-Agent': 'Mozilla/5.0'}
  )
  paragraphs = None

  # read the webpage
  try:
    webpage = urlopen(req, timeout = 10)
    webpage_read = webpage.read()
    webpage_type = webpage.info().get_content_type()

  except HTTPError as error: # if there is a HTTP Error report the error and return no text retrieved
    print('Unable to open webpage: ' + webpage_link + "\nError: " + str(error))
    return None, None

  except URLError as error:
    # if there is a leagacy SSL error reopen the page with a leagacy SSL version (TLS)
    if "SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED"  in str(error):
      print('Open webpage with legacy TLS versions: ' + webpage_link)
      webpage = urlopen(req, context=ctx)
      webpage_read = webpage.read()
      webpage_type = webpage.info().get_content_type()
    else: # if other errors report the error and return no text retrieved
      print('Unable to open webpage: ' + webpage_link + "\nError: " + str(error))
      return None, None

  except TimeoutError as error:
    # if the webpage is not able to load in 10 seconds report a timeout error and return no text retrieved
    print('Timed out on opening webpage: ' + webpage_link + "\nError: " + str(error))
    return None, None

  try:

    # if it is html webpage
    if webpage_type == "text/html":
      webpage = BeautifulSoup(webpage_read, 'html.parser')
      # look for all paragraph sections and change them to pure text form
      paragraphs = [sentences.text.replace("\n", "").replace("\xa0", "") for sentences in webpage.find_all("p")]

    # if it is a pdf webpage
    elif webpage_type == "application/pdf":
      f = io.BytesIO(webpage_read)


      reader = PyPDF2.PdfReader(f)
      paragraphs = ["".join(page.extract_text().splitlines()) for page in reader.pages] # have each page be the seperator

    return paragraphs, webpage_type


  except Exception as error:
      print('Unable to open webpage: ' + webpage_link + "\nError: " + str(error))
      return None, None


Create collections for storing the information scraped from the websites.

In [ ]:
client = chromadb.PersistentClient(path="db")
em = embedding_functions.SentenceTransformerEmbeddingFunction()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# create a collection for all the information gained from the websites
try:
  client.delete_collection(name='WebsitePrioritySummary') # delete the collection so that if you run it again you don't re-add any ids
except:
  pass
priority_collection = client.create_collection(name='WebsitePrioritySummary', embedding_function=em)

try:
  client.delete_collection(name='WebsiteOtherSummary') # delete the collection so that if you run it again you don't re-add any ids
except:
  pass
other_collection = client.create_collection(name='WebsiteOtherSummary', embedding_function=em)

Scrape informations from the website

The information would be stored in corresponding collections (determined by the priority).

In [ ]:
# non-wikipedia websites
# priority links
for webpage_link in  general_priority_articles:
  try:
    paragraphs, webpage_type = get_paragraphs(webpage_link)
  except:
    paragraphs, webpage_type = None, None

  if paragraphs: # makes sure there is a response
    priority_collection.add(
        documents= paragraphs,
        ids=[webpage_link + " <=====> " + str(i) for i in range(len(paragraphs))]
    )

# non-priority links
for webpage_link in  general_other_articles:
  try:
    paragraphs_other, webpage_type = get_paragraphs(webpage_link)
  except:
    paragraphs_other, webpage_type = None, None

  if paragraphs_other: # makes sure there is a response
    # if the webpage type is a pdf then it becomes part of the priority collection
    if webpage_type == "application/pdf":
      priority_collection.add(
          documents= paragraphs_other,
          ids=[webpage_link  + " <=====> " + str(i) for i in range(len(paragraphs_other))]
      )

    else:
      other_collection.add(
          documents= paragraphs_other,
          ids=[webpage_link + " <=====> "  + str(i) for i in range(len(paragraphs_other))]
      )


Unable to open webpage: https://test.history.state.gov/historicaldocuments/frus1952-54v08/d444
Error: HTTP Error 503: Service Temporarily Unavailable
Unable to open webpage: https://2009-2017.state.gov/documents/organization/54374.pdf
Error: HTTP Error 405: Not Allowed
Unable to open webpage: https://www.nato.int/cps/fr/natohq/declassified_181434.htm
Error: HTTP Error 403: Forbidden
Unable to open webpage: https://www.elibrary.imf.org/view/journals/024/1964/001/article-A004-en.xml
Error: HTTP Error 503: Service Unavailable
Unable to open webpage: https://www.tandfonline.com/doi/pdf/10.1080/14794012.2017.1337676
Error: HTTP Error 403: Forbidden
Unable to open webpage: https://www.journals.uchicago.edu/doi/full/10.14318/hau4.2.024
Error: HTTP Error 403: Forbidden
Unable to open webpage: https://www.mfa.gr/missionsabroad/en/un-en/greece-in-organization/peace-and-security.html?page=2
Error: HTTP Error 403: Forbidden
Unable to open webpage: https://www.mfa.gr/en/foreign-policy/
Error: HTTP 

In [ ]:
# wikipedia articles

for wiki_article_title in wikipedia_articles:
  article = wikipedia.WikipediaPage(title = wiki_article_title)
  article_content = article.content.split("\n") # \n separates all the text into paragraphs

  # removes the extra links and all of the citations
  if article_content.count('== See also =='):
    remove_index = article_content.index('== See also ==')
    article_content = article_content[:remove_index]

  # adds the wikipedia information into the priority collection
  priority_collection.add(
      documents= article_content,
      ids=[article.url + " <=====> "  + str(i) for i in range(len(article_content))]
  )


# Final Answer

Using RAG, that is using info collected previously, to answer the problem statement.

In [ ]:
# states the Problem Statement (final question)
final_question = [
    "Hypothesize why" + country_results_text + "gave the UN the gift in " + year_results_text + "?"
]

Find the most applicable text from priority collection text to answer each prompt provided by Google Gemini.

In [ ]:
final_results = []
final_sourecs = set()
# compares the final question to the website info
for prompts in search_prompts:

  result = priority_collection.query(
      query_texts=prompts,
      n_results=3
  )

  final_sourecs.update(set([the_id.split(" <=====> ")[0] for the_id in result["ids"][0]]))
  final_results += result["documents"][0]


Define a function that enhances the visual aspect of the response from Google Gemini

In [ ]:
# function that reformats the response from Google Gemini
def reformat_response(response_text):
  '''reformat_response(response_text) -> str
  returns a string of reformated text'''

  # finds all suppose-to-be-bolded parts of the text and bold them in python
  while response_text.count('**') >= 2:
    # finds the start index of a prompt
    response_text = response_text.replace("**", "\033[1m", 1)



    # finds the end index of a prompt
    response_text = response_text.replace("**", "\033[0m", 1)

  # returns the search prompts
  return response_text

In [ ]:
def folding_line_print(input_string, max_length):
  '''folding_line_print(input_string, max_length) -> None
  Prints the input_string such that each line will have a max length as defined'''

  input_string_list = input_string.splitlines()

  # remove none occurance
  while None in input_string_list:
    input_string_list.remove(None)

  for line in input_string_list:
    while len(line) > max_length:
      # break up the str
      front_part_string = line[:max_length]
      back_part_string = line[max_length:]

      if back_part_string == None:
        back_part_string = ''

      # try to find the cloest word and create a new line after that
      space_pos = front_part_string[::-1].find(' ')

      if space_pos != -1:
        space_pos = max_length - space_pos

        print(front_part_string[:space_pos])

        line = front_part_string[space_pos:] + back_part_string
      else:
        print(front_part_string) + back_part_string

    else:
      if type(line) == str:
        print(line)



Use RAG to answer each of the provided text with the problem statement. Then ask Google Gemini to summerize the responses provided. This answer is printed as the final response.

In [ ]:
# iterates through all of the text collected by the collection
for n in range(len(final_results)):
    responses = chat_session.send_message(f'Given {final_results[n]}, answer {final_question}') # for each text ask Google Gemini to use the text to answer the problem statement


final_responses = chat_session.send_message("Summarize the "+ str(len(final_results)) + " responses above to answer" + final_question[0]) # summarize the answers provided by Google Gemini to form one coherent response

In [ ]:
# prints the final response
print(reformat_response(final_responses.text) + '\n')


print("\033[1mSources Used For This Response:\033[0m")
for link in final_sourecs:
  print("* " + link)

The analysis of the historical context, particularly the US-Greece relationship during the Cold War, reveals that the Poseidon 
statue gift in 1953 was a strategic move by Greece driven by multiple factors:

* Gratitude for US Aid: Greece was deeply indebted to the US for its support during and after the Greek Civil War 
(1946-1949). The gift was a public expression of gratitude for this crucial assistance.
* Strategic Importance:  Greece's location made it a vital buffer against Soviet expansion in the Eastern Mediterranean. 
The gift served as a reminder of its strategic importance to the US and its commitment to the Western alliance.
* Counteracting Anti-American Sentiment:  The gift could have been a preemptive measure to address potential 
anti-American sentiment in Greece, which was brewing due to US influence and Cold War policies.
* Public Relations Coup:  The gift was a public relations coup, showcasing Greece's cultural heritage and its dedication 
to international cooperatio